In [ ]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch

# 确保PyTorch能够检测到GPU

In [ ]:

import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [ ]:

# Load pretrained LLM
QWEN2d5_7B_INST = "/data/liangyunfei/Qwen2.5-7B-Instruct"
QWEN2d5_32B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ"
QWEN2d5_72B_INST_AWQ = "/data/liangyunfei/Qwen2.5-72B-Instruct-AWQ"

#model_7Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
#tokenizer_7Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

model_32Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
tokenizer_32Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")



In [ ]:

from transformers import TextStreamer

streamer = TextStreamer(tokenizer_32Bn, skip_prompt=True, skip_special_tokens=True)

In [ ]:

sys_prompt1 = """
    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.
"""
sys_prompt1 = '你是一个非常专业的糖尿病患者照护师，请专注于给对方提供情绪价值，输出用简短的话语'

def message_to_response_via_LLM(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01, streamer=streamer)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response
def message_to_response_no_stream(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
sys_prompt2 = '对于空腹血糖，一般认为正常范围是3.9～6.1mmol/l。餐后血糖的正常值则有所不同，餐后1小时血糖通常在6.7-9.4mmol/l之间，最多不超过11.1mmol/l。餐后2小时血糖应小于7.8mmol/l，而餐后3小时血糖应恢复到正常水平，即小于7.8mmol/l。此外，孕妇的血糖正常值有所不同。孕妇空腹血糖不应超过5.1mmol/l，餐后1小时血糖不应超过10.0mmol/l，餐后2小时血糖不应超过8.5mmol/l。'
format_prompt = "如果从中可以判断血糖异常，仅仅输出‘@照护师’，否则输出‘是’"

res_messages = [
        
        {"role": "system", "content": sys_prompt1},
    ]
res_messages2 = [        
        {"role": "system", "content": sys_prompt2},
    ]

sys_prompt3 = ''
format_prompt3 = "如果对方在问某种食物是否能吃类似的问题，仅仅输出‘@照护师’，否则输出‘否’"
res_messages3 = [
        {"role": "system", "content": sys_prompt3},
    ]

answer = ''
answer_judge1 = ''
answer_judge = ''
while True:
    query_prompt = input()#input()#"我吃的是咖喱饭"
    print('患者：')
    print(query_prompt)
    res_messages2.append({"role": "system", "content": answer_judge})
    res_messages2.append({"role": "system", "content": query_prompt + format_prompt})
    answer_judge = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages2, max_new_tokens=100)
    print('医生：')
    if '是' not in answer_judge:
        print(answer_judge)
        print('@照护师')

    res_messages.append({"role": "system", "content": answer})
    res_messages.append({"role": "system", "content": query_prompt})
    answer = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
    else:
        res_messages3.append({"role": "system", "content": answer_judge})
        res_messages3.append({"role": "system", "content": query_prompt + format_prompt3})
        answer_judge3 = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages3, max_new_tokens=100)
        if '否' not in answer_judge3:
            print('@照护师')

In [ ]:
sys_prompt2 = '对于空腹血糖，一般认为正常范围是3.9～6.1mmol/l。餐后血糖的正常值则有所不同，餐后1小时血糖通常在6.7-9.4mmol/l之间，最多不超过11.1mmol/l。餐后2小时血糖应小于7.8mmol/l，而餐后3小时血糖应恢复到正常水平，即小于7.8mmol/l。此外，孕妇的血糖正常值有所不同。孕妇空腹血糖不应超过5.1mmol/l，餐后1小时血糖不应超过10.0mmol/l，餐后2小时血糖不应超过8.5mmol/l。'
format_prompt = "如果从中可以判断血糖异常，仅仅输出‘@照护师’，否则输出‘是’"
res_messages2 = [
        
        {"role": "system", "content": sys_prompt2},
    ]
answer = ''
answer_judge = ''
while True:
    
    query_prompt = input()#input()#"我吃的是咖喱饭"
    print(query_prompt)
    res_messages2.append({"role": "system", "content": answer_judge})
    res_messages2.append({"role": "system", "content": query_prompt + format_prompt})
    answer_judge = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages2, max_new_tokens=100)
    print(answer_judge)

In [ ]:
sys_prompt3 = ''
format_prompt = "如果对方在问某种食物是否能吃类似的问题，仅仅输出‘@照护师’，否则输出‘否’"
res_messages3 = [
        
        {"role": "system", "content": sys_prompt2},
    ]
answer = ''
while True:
    
    query_prompt = input()#input()#"我吃的是咖喱饭"
    print(query_prompt)
    res_messages3.append({"role": "system", "content": answer_judge})
    res_messages3.append({"role": "system", "content": query_prompt + format_prompt})
    answer_judge1 = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages3, max_new_tokens=100)
    print(answer_judge)

In [ ]:
sys_prompt2 = '对于空腹血糖，一般认为正常范围是3.9～6.1mmol/l。餐后血糖的正常值则有所不同，餐后1小时血糖通常在6.7-9.4mmol/l之间，最多不超过11.1mmol/l。餐后2小时血糖应小于7.8mmol/l，而餐后3小时血糖应恢复到正常水平，即小于7.8mmol/l。此外，孕妇的血糖正常值有所不同。孕妇空腹血糖不应超过5.1mmol/l，餐后1小时血糖不应超过10.0mmol/l，餐后2小时血糖不应超过8.5mmol/l。'
format_prompt = "如果从中可以判断血糖异常，仅仅输出‘@照护师’，否则输出‘是’"
res_messages2 = [    
        {"role": "system", "content": sys_prompt2},
    ]
answer = ''
answer_judge = ''
query_prompt = input()#input()#"我吃的是咖喱饭"
print(query_prompt)
res_messages2.append({"role": "system", "content": answer_judge})
res_messages2.append({"role": "system", "content": query_prompt + format_prompt})
answer_judge = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages2, max_new_tokens=100)
print(answer_judge)

In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn, json, datetime
app = FastAPI()

class Query(BaseModel):
    text: str



@app.post("/chat/")
async def chat(query: Query):
    sys_prompt2 = '对于空腹血糖，一般认为正常范围是3.9～6.1mmol/l。餐后血糖的正常值则有所不同，餐后1小时血糖通常在6.7-9.4mmol/l之间，最多不超过11.1mmol/l。餐后2小时血糖应小于7.8mmol/l，而餐后3小时血糖应恢复到正常水平，即小于7.8mmol/l。此外，孕妇的血糖正常值有所不同。孕妇空腹血糖不应超过5.1mmol/l，餐后1小时血糖不应超过10.0mmol/l，餐后2小时血糖不应超过8.5mmol/l。'
    format_prompt = "如果从中可以判断血糖异常，仅仅输出‘@照护师’，否则输出‘是’"
    res_messages2 = [    
            {"role": "system", "content": sys_prompt2},
        ]
    answer = ''
    answer_judge = ''
    query_prompt = input()#input()#"我吃的是咖喱饭"
    print(query_prompt)
    res_messages2.append({"role": "system", "content": answer_judge})
    res_messages2.append({"role": "system", "content": query_prompt + format_prompt})
    answer_judge = message_to_response_no_stream(model_32Bn, tokenizer_32Bn, res_messages2, max_new_tokens=100)
    print(answer_judge)
    return {"result": answer_judge}

uvicorn.run(app, host="0.0.0.0", port=6667)

In [ ]:
from transformers import BertModel, BertConfig
import torch.nn as nn
from transformers import BertTokenizer

# 加载预训练的BERT模型配置
config = BertConfig.from_pretrained('/data/liangyunfei/bert-base-uncased')

# 定义自定义的BERT模型结构
class CustomBertModel(nn.Module):
    def __init__(self, config):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 1)  # 假设这是一个回归任务，输出一个值

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        pooled_output = outputs[1]  # 获取[CLS]标记的输出
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 初始化自定义BERT模型
model = CustomBertModel(config)


: 

In [5]:
X.shape

(1, 100, 5)

In [7]:
X_tensor.shape

torch.Size([1, 100, 5])

In [9]:
y_tensor.shape

torch.Size([1, 100])

In [ ]:
list('0')

['d', 's', 'a', 'd', 's', 'a']